# Studying News Popularity in terms of Number of Shares

Courtesy of K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. Proceedings of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence, September, Coimbra, Portugal.

Refer to https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity for details. 

** Table of Content**

1. [Set Up Workspace](#Set-Up-Workspace)
2. [Define preprocessing script for model training](#Define-preprocessing-script-for-model-training)
3. [Specify experiment settings and compute targets](#Specify-Experiment-Settings-and-Compute-Targets)
4. [Submit an experiment](#Submit-an-experiment)
5. [Explain Model](#Explain-Model)
6. [Register Model](#Register-Model)


Tip: if you need to debug your SDK, you can run this line `from azureml._logging.debug_mode import debug_sdk` and call `debug_sdk()` just before the code block you want to debug.

# Set Up Workspace

In [1]:
import os
import pandas as pd
import numpy as np
import logging

## Read data from a website
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

## Split data 
from sklearn.model_selection import train_test_split


## Azure-related
import azureml.core
print("SDK version:", azureml.core.VERSION)
from azureml.core import Workspace, Experiment, Run

### Specify compute targets
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget


SDK version: 1.0.33


In [2]:
ws = Workspace.from_config()

In [3]:
print("This notebook was validated with the AzureML SDK version 1.0.33. You are currently using ", azureml.core.VERSION)

This notebook was validated with the AzureML SDK version 1.0.33. You are currently using  1.0.33


# Define preprocessing script for model training

If you intend to use remote compute, rather than local compute, you should provide your data in a python script as shown below, rather than a series of jupyter cells. If you were to use local compute, you can write your codes as you usually would in a jupyter notebook. An example would be shown later in Section [Specify Experiment Settings and Compute Targets](#Specify-Experiment-Settings-and-Compute-Targets) to show you how you can supply objects to use either remote or local compute.

In [4]:
import os
project_folder = os.getcwd()

In [4]:
%%writefile $project_folder/preprocess.py
import os
project_folder = os.getcwd()
print(project_folder)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Read data from a website
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

def get_data():
    # Read Data from url
    print('Reading data...')
    resp = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip")
    zipfile = ZipFile(BytesIO(resp.read()))
    zipfile.namelist()
    file = 'OnlineNewsPopularity/OnlineNewsPopularity.csv'
    df = pd.read_csv(zipfile.open(file))
    
    # Preprocessing
    # Remove beginning white space in the columns
    print('Stripping off white space...')
    df.rename(columns=lambda x: x.strip(), inplace=True)
    
    # Set Target Label
    # Define number of popularity categories to predict
    print('Make target categories')
    share_categories = [1,2,3,4,5]
    df['share_cat'] = np.array(pd.qcut(df['shares'], 5, share_categories))
    df['share_cat'].dtype
    df['share_cat'] = np.array(df['share_cat'].astype('category'))
    
    # Split Data
    # time delta and url are not predictive attributes, exclude them
    x_df = df[df.columns[2:-2]] # url and time delta are the first two attributes 
    y_df = df[df.columns[-1]]
    
    print('Splitting data...')
    x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, 
                                                        random_state=607)
    
    return { "X": x_train.values, "y": y_train.values, 
            "X_valid": x_test.values, "y_valid": y_test.values}


Overwriting /home/nbuser/library/preprocess.py


### Data Understanding
This section aims only to give a glimpse into the data set. Writing preprocess.py above is the only script needed to submit a model training job

In [5]:
project_folder = os.getcwd()
data_folder = os.path.join(os.getcwd(), 'data/OnlineNewsPopularity')
print(data_folder)
os.makedirs(data_folder, exist_ok=True)

/home/nbuser/library/data/OnlineNewsPopularity


In [6]:
resp = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip")
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()
file = 'OnlineNewsPopularity/OnlineNewsPopularity.csv'
original_df = pd.read_csv(zipfile.open(file))

df = original_df
df.shape

(39644, 61)

We are inspecting the data frame to make sure that the data looks correct. Note that the first and second columns are not predictive, hence they are not included in the training data set as predictors. The url column gives the URL of the article whereas `timedelta` is the number of days between the article publication and the dataset acquisition. For the complete list of description of each columns, refer to this [page](https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity#).

In [7]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


# Specify Experiment Settings and Compute Targets

Specify a compute target based on an existing cluster or you can create a new compute target. You can view your existing clusters here 
<br>
<img src="Images/compute_target.png" width="2000">:

In [7]:
# gpucluster is an existing compute target
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

compute_target = ws.compute_targets[compute_name]

By using automated machine learning, Azure iterates through appropriate machine learning algorithms depending on your task. The array of models supported is listed [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train). Notice that in the settings below, preprocess flag is set to True. Refer to this [article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train?view=azure-dataprep-py#data-pre-processing-and-featurization) to learn what preprocessing steps Azure takes.


In [8]:
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 10,
    "primary_metric" : 'AUC_weighted',
    "verbosity" : logging.INFO,
    "preprocess": True
}

You can specify your experiment settings as such. Since I am interested in using remote compute target, I am turning the flag `local_compute` to be False. Note the difference between two settings in terms of how you can supply the objects. Beyond what's specified as a parameter in the settings, you can also provide other parameters according to your needs as well. You can refer to [this web page](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py) to read about other parameters.

In [9]:
from azureml.train.automl import AutoMLConfig

local_compute = False

if local_compute: 
    print('using local compute')
    automated_ml_config = AutoMLConfig(task = 'classification',
                                 debug_log = 'automated_ml_errors.log',
                                 compute_target=compute_target,
                                 path = project_folder,
                                 X = x_train.values,
                                 y = y_train_array,
                                 X_valid = x_test.values,
                                 **automl_settings)
else: 
    print('using remote compute')
    automated_ml_config = AutoMLConfig(task = 'classification',
                                 debug_log = 'automated_ml_errors.log',
                                 compute_target=compute_target,
                                 path = project_folder,
                                 data_script= project_folder + "/get_data.py",
                                 model_explainability=True,
                                 **automl_settings)

using remote compute


# Submit an experiment

Now that you have defined your experiment settings above, you can create and name the experiment to run in the workspace you desired. 
<br>
<img src="Images/experiment_homepage.png" width="1500">

In [10]:
# create an experiment
experiment = Experiment(workspace = ws, name = "news_popularity")

Next, submit the experiment. If you wish to view the output within the notebook, turn on the flag `show_output`.

In [ ]:
# submit an experiment
run = experiment.submit(automated_ml_config, show_output=True)

/home/nbuser/library
Reading data...
Stripping off white space...
Make target categories
Splitting data...
Running on remote compute: gpucluster
Parent Run ID: AutoML_2e02b696-bf8a-47f4-b035-64347532b27e
********************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
SAMPLING %: Percent of the training data to sample.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
********************************************************************************************************************

 ITERATION   PIPELINE                                       SAMPLING %  DURATION      METRIC      BEST
         0   StandardScalerWrapper SGD                      100.0000    0:26:46       0.6304    0.6304


Different runs of the same experiment name will be grouped under the same parent run.

<br>
<img src="Images/experiment_runs.png" width="1000">

In [17]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

<img src="Images/run_output_details.png" width="1500">

###  Retrieve the Best Model

You can retrieve the best model based on the run that you just defined above.

In [76]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: news_popularity,
Id: AutoML_96d26b44-876a-4643-b56f-6250e616e123_29,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
               estimators=[('LightGBM', Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ...666666666667, 0.06666666666666667, 0.26666666666666666, 0.13333333333333333, 0.26666666666666666]))])


### Best Model Based on Any Other Metric

Even though I specified `AUC_weighted` to be the metric that I wanted to measure against, I can also choose other relevant metrics to pick the best model. In the following cell, I am interested in looking up the model that has the highest accuracy rate.

In [78]:
lookup_metric = "accuracy"
best_run, fitted_model = run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: news_popularity,
Id: AutoML_96d26b44-876a-4643-b56f-6250e616e123_29,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
               estimators=[('LightGBM', Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ...666666666667, 0.06666666666666667, 0.26666666666666666, 0.13333333333333333, 0.26666666666666666]))])


## To retrieve the best model run without re-training

There are also times when you wish to retrieve the best model run, without having to re-define your model objects again. In this scenario, you can head to the home page of your desired experiment to obtain the `Run Id` of the experiment.
<br>
<img src="Images/run_id.png" width="1500">

In [11]:
from azureml.core import get_run
run_cpu_id = 'AutoML_2e02b696-bf8a-47f4-b035-64347532b27e_9'
# run_cpu_id = 'AutoML_fd9055d1-1f4b-4484-ae70-d773ca82bd67' #get from portal
best_run = get_run(experiment, run_cpu_id)

# Explain Model

Model explanability is important for any model users to understand why the model gives a particular outcome. Azure incorporates this explanability option in `automated_ml_config` settings easily for people who want to retrieve model explanation while using the `automl` functionality. 

Since I included the flag `model_explanability` in my `automated_ml_config` settings (refer to the last cell under Section [Specify Experiment Settings and Compute Targets](#Specify-Experiment-Settings-and-Compute-Targets)], I can easily retrieve model explanation. To learn about the `automlexplainer` module and the objects returned by `retrieve_model_explanation`, you can visit this [page](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlexplainer?view=azure-ml-py) to gain more information. `shap_values` below returns a matrix of feature importance values. The dimension of this matrix is (# examples x # features).

In [12]:
# if there is problem in importing `numpy.core.multiarray`, upgrade your numpy package
# !pip install -U numpy
from azureml.train.automl.automlexplainer import retrieve_model_explanation

## for a specific run 
shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = \
    retrieve_model_explanation(best_run)

You can inspect overall feature importance and their importance values as shown below. `overall_imp` shows features in decreasing importance whereas `overall_summary` gives the importance values of the associated features. If you are interested in class-level importance features and values, you can print `per_class_imp` and `per_class_summary` to take a look.

In [15]:
print(overall_imp)
print(overall_summary)

['C26_MeanImputer', 'C27_MeanImputer', 'C29_MeanImputer', 'C25_MeanImputer', 'C37_MeanImputer', 'C38_MeanImputer', 'C42_MeanImputer', 'C13_MeanImputer', 'C40_MeanImputer', 'C16_MeanImputer', 'C24_MeanImputer', 'C6_MeanImputer', 'C39_MeanImputer', 'C18_MeanImputer', 'C3_MeanImputer', 'C41_MeanImputer', 'C8_MeanImputer', 'C43_MeanImputer', 'C20_MeanImputer', 'C17_MeanImputer', 'C21_MeanImputer', 'C28_MeanImputer', 'C50_MeanImputer', 'C22_MeanImputer', 'C5_MeanImputer', 'C23_MeanImputer', 'C15_MeanImputer', 'C48_MeanImputer', 'C44_MeanImputer', 'C2_MeanImputer', 'C19_MeanImputer', 'C14_MeanImputer', 'C10_MeanImputer', 'C55_MeanImputer', 'C7_MeanImputer', 'C11_MeanImputer', 'C45_MeanImputer', 'C35_MeanImputer', 'C47_MeanImputer', 'C56_MeanImputer', 'C49_MeanImputer', 'C32_MeanImputer', 'C58_MeanImputer', 'C52_MeanImputer', 'C46_MeanImputer', 'C53_MeanImputer', 'C57_MeanImputer', 'C34_MeanImputer', 'C1_MeanImputer', 'C51_MeanImputer', 'C36_MeanImputer', 'C33_MeanImputer', 'C9_MeanImputer', 

<br>
The block above will also create a folder within your project directory to save `explanation` in json files.
<br>
<img src="Images/explanation.png" width="1500">

If you would like to retrieve feature importance chart, go to your portal and click on the experiment run you are interested in to view it. 
<br>
<img src="Images/explanation_chart.png" width="500">

# Register Model

Finally, if you are interested in hosting your model online, you can register your model! Now, you can go on to deploy your model and connect your webpage with your model or do batch scoring if you would like to! 

Typically, deployment follows three steps:

1. Register model (shown here below!)
2. Deploy model
3. Test your deployed model

You can refer [this page](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where) to learn how to deploy your model to production locally or on a specific cluster/compute target.

In [36]:
model = best_run.register_model('predict_news_popularity')

<br>
<img src="Images/register_model.png" width="1000">